# Assuming 2 sets of point-clouds, we will compute the MMD, Coverage and JSD as done in the paper.

(To compute these metrics you __don't need__ to have tflearn installed, only the structural: EMD, Chamfer losses and sklearn for the JSD.)

In [ ]:
from tqdm import trange
import sys
sys.path.append('/latent_3d')

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [1]:
import numpy as np
import os.path as osp

from latent_3d_points.src.evaluation_metrics import minimum_mathing_distance, \
jsd_between_point_cloud_sets, coverage

from latent_3d_points.src.in_out import snc_category_to_synth_id,\
                                        load_all_point_clouds_under_folder

ImportError: No module named latent_3d_points.src.evaluation_metrics

Load some point-clouds and make two sets (sample_pcs, ref_pcs) from them. The ref_pcs is considered as the __ground-truth__ data while the sample_pcs corresponds to a set that is matched against it, e.g. comes from a generative model.

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
top_in_dir = '../data/shape_net_core_uniform_samples_2048/' # Top-dir of where point-clouds are stored.
class_name = raw_input('Give me the class name (e.g. "chair"): ').lower()
syn_id = snc_category_to_synth_id()[class_name]
class_dir = osp.join(top_in_dir , syn_id)
all_pc_data = load_all_point_clouds_under_folder(class_dir, n_threads=8, file_ending='.ply', verbose=True)

Give me the class name (e.g. "chair"): chair
6778 pclouds were loaded. They belong in 1 shape-classes.


In [4]:
n_ref = 100 # size of ref_pcs.
n_sam = 150 # size of sample_pcs.
all_ids = np.arange(all_pc_data.num_examples)
ref_ids = np.random.choice(all_ids, n_ref, replace=False)
sam_ids = np.random.choice(all_ids, n_sam, replace=False)
ref_pcs = all_pc_data.point_clouds[ref_ids]
sample_pcs = all_pc_data.point_clouds[sam_ids]

Compute the three metrics.

In [11]:
ae_loss = 'chamfer'  # Which distance to use for the matchings.

if ae_loss == 'emd':
    use_EMD = True
else:
    use_EMD = False  # Will use Chamfer instead.
    
batch_size = 100     # Find appropriate number that fits in GPU.
normalize = True     # Matched distances are divided by the number of 
                     # points of thepoint-clouds.

mmd, matched_dists = minimum_mathing_distance(sample_pcs, ref_pcs, batch_size, normalize=normalize, use_EMD=use_EMD)

cov, matched_ids = coverage(sample_pcs, ref_pcs, batch_size, normalize=normalize, use_EMD=use_EMD)

jsd = jsd_between_point_cloud_sets(sample_pcs, ref_pcs, resolution=28)

In [12]:
print mmd, cov, jsd

0.0714721 0.73 0.0396569736382


For a detailed breakdown of the evaluation functions, inspect their docs.

In [24]:
print coverage.__doc__
print minimum_mathing_distance.__doc__
print jsd_between_point_cloud_sets.__doc__

Computes the Coverage between two sets of point-clouds.

    Args:
        sample_pcs (numpy array SxKx3): the S point-clouds, each of K points that will be matched
            and compared to a set of "reference" point-clouds.
        ref_pcs    (numpy array RxKx3): the R point-clouds, each of K points that constitute the
            set of "reference" point-clouds.
        batch_size (int): specifies how large will the batches be that the compute will use to
            make the comparisons of the sample-vs-ref point-clouds.
        normalize  (boolean): When the matching is based on Chamfer (default behavior), if True,
            the Chamfer is computed as the average of the matched point-wise squared euclidean
            distances. Alternatively, is their sum.
        use_sqrt  (boolean): When the matching is based on Chamfer (default behavior), if True,
            the Chamfer is computed based on the (not-squared) euclidean distances of the matched
            point-wise euclid